<a href="https://colab.research.google.com/github/ayanyag/Complete-Python-3-Bootcamp/blob/master/EP_EDE_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Overview and Requirements

**Notebook Overview and Requirements**

This Google Colab notebook automates the process of generating and validating a regulatory XBRL report. Specifically, this script is configured to update and validate the Financial information on the safeguarding system reports:

- EP_4-1: Salvaguarda de fondos de usuarios de servicios de pago y dinero electrónico [7510]

- EP_4-2: Cuentas de salvaguarda (a) [7511]

- EP_4-3: Detalle de cuentas de salvaguarda (a) [7512]

The workflow reads new data from an Excel file, updates a template XBRL instance, validates the result against the official taxonomies, and packages the final report into a zip archive for submission.


**Before You Begin**

Please ensure you have the following files prepared and in the correct locations:

1. In your Google Drive (in the folder specified in the script, e.g., /MyDrive/BDE_REPORTING_SCRIPT/XBRL_Taxonomies/):

    - All necessary taxonomy packages as .zip files (e.g., es-bde.zip, Full_taxonomy_release_4.1.zip, etc.). These are stored in Drive to avoid re-uploading large files every session.

2. Ready to upload to this Colab session:

    - The source Excel file containing the data for the current reporting period (e.g., EP_4_MAY25.xlsx).
    - The base XBRL instance file from the previous period, which will be used as a template (e.g., ES6885_..._20250430.xbrl).

**How to Use**

Execute the cells in this notebook from top to bottom. The initial cells perform a one-time setup for the session (installing tools and connecting to your Drive), and the final cells carry out the report generation and validation. Review the output of the final cell to confirm the validation was successful.





# Install external software and Python libraries

In [1]:
# --- Install Necessary Libraries (Run Once Per Session) ---

"""
    This cell prepares the Google Colab environment by installing all necessary
    external software and Python libraries required for the entire workflow.

    It performs the following actions:
      1. Installs 'p7zip-full': A command-line utility needed to decompress
          .7z archive files, which some of the taxonomy packages use.
      2. Installs Python Libraries: Uses 'pip' to install pandas, openpyxl,
          and python-dateutil for data manipulation and reading Excel files.
      3. Installs Arelle: Fetches and installs the Arelle XBRL processor
          directly from its official GitHub repository to ensure access to the
          command-line tool.

    USAGE:
    This cell should be run ONLY ONCE at the beginning of each new Colab session,
    as the environment is reset every time it is disconnected.
    """

print("Installing libraries...")
!sudo apt-get -y install p7zip-full
!pip install openpyxl python-dateutil pandas -q
!pip install openpyxl python-dateutil pandas lxml -q
!pip install git+https://github.com/Arelle/Arelle.git -q
print("\n✅ Libraries installed successfully.")
print("\nYou can now run the main script in the next cell.")

Installing libraries...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 3.6 MB/s eta 0:00:00

✅ Libraries installed successfully.

You can now run the main script in the next cell.


# Mount Google Drive

In [2]:
# --- Mount Google Drive (Run Once Per Session) ---

"""
    This cell mounts the user's personal Google Drive to the Colab environment,
    making its contents accessible to the script.

    WHY IT'S NECESSARY:
    This connection is essential for the script to access the large XBRL
    taxonomy packages (.zip files). Storing these files in Google Drive is
    efficient because it avoids the need to upload them every session.

    IMPORTANT USER INTERACTION:
    When you run this cell, you will be prompted with a URL. You must:
      1. Click the link.
      2. Sign in to your Google account.
      3. Grant permission for Colab to access your Drive.
      4. Copy the authorization code that Google provides.
      5. Paste that code back into the input box in this cell and press Enter.

    USAGE:
    This cell should be run once per session, after the libraries in Cell 1
    have been installed.
    """

from google.colab import drive
print("Connecting to Google Drive...")
drive.mount('/content/drive')
print("\n✅ Google Drive mounted successfully at /content/drive.")

Connecting to Google Drive...
Mounted at /content/drive

✅ Google Drive mounted successfully at /content/drive.


# Import libraries

In [3]:
# --- Import Libraries (Run Once Per Session) ---

"""
    This cell imports all the necessary Python libraries and modules into the
    current Colab session's memory.

    By placing all imports in a separate cell, we ensure that these tools are
    loaded only once, which makes re-running the main script faster and keeps
    the overall notebook more organized.

    Key Libraries Imported:
      - xml.etree.ElementTree: For creating, parsing, and writing XML/XBRL files.
      - openpyxl & pandas: For reading and handling data from the .xlsx files.
      - re: For using regular expressions to find codes within cell text.
      - datetime, dateutil, calendar: For all date-related logic, like calculating
        the next reporting month.
      - zipfile, os, subprocess: For file system operations like creating folders,
        unzipping archives, and running external command-line tools.
      - arelle: Specific classes imported from the Arelle XBRL processor for
        validation tasks.

    USAGE:
    This cell should be run once per session, after Cell 1 (Installation) and
    Cell 2 (Drive Mount) have completed successfully.
    """

import xml.etree.ElementTree as ET
from lxml import etree as ET
import openpyxl
import pandas as pd
import re
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import calendar
import zipfile
import os
import subprocess
from arelle import Cntlr
from arelle.FileSource import FileSource

print("✅ Libraries imported successfully. Ready for the next steps.")

✅ Libraries imported successfully. Ready for the next steps.


# Pre-flight check

In [4]:
# --- File Explorer & Pre-flight Check (Run Before Main Script) ---

"""
    This cell acts as a 'File Explorer' for your Google Drive. It lists all
    files within your specified taxonomy directory and shows their size and last
    modified date.

    WHY IT'S NECESSARY:
    This is an essential pre-flight check. By running this cell, you can:
      1. Visually confirm that all required taxonomy packages are present.
      2. Quickly check the 'Last Modified' date of your files and compare them
        with the latest versions available on the official regulator websites
        to decide if you need to download updates.

    HOW TO USE:
    Run this cell after mounting your Google Drive to audit your files.
    """

print("--- Listing contents of your Google Drive taxonomy folder... ---")

# This path should exactly match the 'drive_path' variable in the next cell.
drive_path = "/content/drive/MyDrive/BDE_REPORTING_SCRIPT/XBRL_Taxonomies/"

if not os.path.isdir(drive_path):
    print(f"❌ ERROR: The directory was not found: {drive_path}")
    print("Please ensure your Google Drive is mounted and the path is correct.")
else:
    try:
        # Get a list of all files in the directory
        files_in_drive = os.listdir(drive_path)

        if not files_in_drive:
            print("The folder exists but is empty.")
        else:
            print(f"\nFound {len(files_in_drive)} files in '{drive_path}':")
            # Loop through each file and print its details
            for filename in sorted(files_in_drive):
                full_path = os.path.join(drive_path, filename)
                # Get file size and convert to MB
                size_in_mb = os.path.getsize(full_path) / (1024 * 1024)
                # Get last modified timestamp and format it
                mod_time = os.path.getmtime(full_path)
                mod_date_str = datetime.fromtimestamp(mod_time).strftime('%Y-%m-%d %H:%M:%S')

                print(f"  - File: {filename:<35} | Size: {size_in_mb:>5.2f} MB | Last Modified: {mod_date_str}")

    except Exception as e:
        print(f"\nAn error occurred while listing files: {e}")

--- Listing contents of your Google Drive taxonomy folder... ---

Found 5 files in '/content/drive/MyDrive/BDE_REPORTING_SCRIPT/XBRL_Taxonomies/':
  - File: Full_taxonomy_release_4.1.zip       | Size: 102.52 MB | Last Modified: 2025-06-19 09:51:48
  - File: es-bde-aux.zip                      | Size:  0.07 MB | Last Modified: 2025-06-19 09:42:35
  - File: es-bde.zip                          | Size: 63.57 MB | Last Modified: 2025-06-19 10:58:56
  - File: eu-eurofiling.zip                   | Size:  0.16 MB | Last Modified: 2025-06-19 09:39:57
  - File: srf-eac_full_taxonomy_0.zip         | Size: 12.14 MB | Last Modified: 2025-06-19 09:44:15


# Local taxonomy enviroment

In [5]:
# --- Prepare Local Taxonomy Environment ---

"""
    This cell is responsible for preparing the complete local taxonomy environment.
    It takes all the required taxonomy packages (which can be a mix of standard
    .zip and .7z formats) from Google Drive and extracts them into a single,
    structured folder within the Colab session.

    KEY LOGIC:
    1. Efficiency Check: It first checks if the environment has already been
      prepared by looking for the existence of all key sub-folders (e.g.,
      'www.bde.es', 'www.eba.europa.eu', etc.). If they all exist, it skips
      the time-consuming extraction process.
    2. Smart Decompression: If the environment is not ready, it iterates
      through each archive. It intelligently detects its format (standard Zip
      or 7-Zip) and uses the correct tool to decompress it.

    WHY IT'S NECESSARY:
    This step creates a comprehensive local mirror of all required taxonomies.
    This allows the final validation script (in the next cell) to run in a
    true offline mode using URL remapping, which is essential to bypass the
    "403 Forbidden" network errors.

    USAGE:
    Run this cell once per session after installing libraries (Cell 1) and
    mounting Google Drive (Cell 2). The successful completion of this cell is a
    prerequisite for running the main script.
    """

print("--- Preparing Local Taxonomy Environment ---")

# --- Define File Paths ---
drive_path = "/content/drive/MyDrive/BDE_REPORTING_SCRIPT/XBRL_Taxonomies/"
taxonomy_zip_files = [
    drive_path + 'es-bde.zip',
    drive_path + 'es-bde-aux.zip',
    drive_path + 'eu-eurofiling.zip',
    drive_path + 'srf-eac_full_taxonomy_0.zip',
    drive_path + 'Full_taxonomy_release_4.1.zip'
]
local_taxonomy_folder = 'local_taxonomies'

# --- Define all subfolders that should exist after a full extraction ---
# These correspond to the URL remapping rules in the main script.
expected_subfolders = [
    os.path.join(local_taxonomy_folder, 'www.bde.es'),
    os.path.join(local_taxonomy_folder, 'www.eba.europa.eu'),
    os.path.join(local_taxonomy_folder, 'www.eurofiling.info'),
    os.path.join(local_taxonomy_folder, 'EBA_XBRL_4.0_Severity_4.0.0.1'),
    os.path.join(local_taxonomy_folder, 'www.srb.europa.eu')
]

# --- NEW: Check if ALL expected subfolders already exist ---
is_prepared = all(os.path.isdir(path) for path in expected_subfolders)

if is_prepared:
    print(f"✅ Local taxonomy environment already exists at '{local_taxonomy_folder}'. Skipping extraction.")

else:
    # If the check path doesn't exist, run the full extraction process.
    print(f"Local taxonomy environment not found. Starting full extraction process...")
    try:

        # --- Create local folder and unzip all packages ---
        print(f"\n--- Unzipping all taxonomy packages ---")
        os.makedirs(local_taxonomy_folder, exist_ok=True)

        files_unzipped = 0
        for zip_file_path in taxonomy_zip_files:
            if not os.path.exists(zip_file_path):
                print(f"WARNING: Could not find file at '{zip_file_path}'")
                continue

            print(f"Processing: {os.path.basename(zip_file_path)}...")
            result = subprocess.run(['file', zip_file_path], capture_output=True, text=True)
            file_type_info = result.stdout

            if '7-zip archive' in file_type_info:
                print(f"  -> Detected as 7-Zip archive. Extracting with 7z...")
                os.system(f'7z x "{zip_file_path}" -o"{local_taxonomy_folder}" -y > /dev/null')
                files_unzipped += 1
            elif 'Zip archive' in file_type_info:
                print(f"  -> Detected as standard Zip archive. Extracting...")
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(local_taxonomy_folder)
                files_unzipped += 1
            else:
                print(f"  -> WARNING: Could not determine archive type for {os.path.basename(zip_file_path)}. Skipping.")

        if files_unzipped < len(taxonomy_zip_files):
            print(f"\nWARNING: Only {files_unzipped} out of {len(taxonomy_zip_files)} packages were unzipped.")
        else:
            print(f"\n✅ Local taxonomy environment is now ready. All {files_unzipped} packages unzipped.")

    except Exception as e:
        print(f"\nAn unexpected error occurred during unzipping: {e}")

--- Preparing Local Taxonomy Environment ---
Local taxonomy environment not found. Starting full extraction process...

--- Unzipping all taxonomy packages ---
Processing: es-bde.zip...
  -> Detected as 7-Zip archive. Extracting with 7z...
Processing: es-bde-aux.zip...
  -> Detected as 7-Zip archive. Extracting with 7z...
Processing: eu-eurofiling.zip...
  -> Detected as standard Zip archive. Extracting...
Processing: srf-eac_full_taxonomy_0.zip...
  -> Detected as standard Zip archive. Extracting...
Processing: Full_taxonomy_release_4.1.zip...
  -> Detected as standard Zip archive. Extracting...

✅ Local taxonomy environment is now ready. All 5 packages unzipped.


# Generate XBRL report

In [8]:
# --- Main Script to Generate, Validate, and Package XBRL Report ---

def main():
    """
    This script automates the monthly generation and validation of a regulatory XBRL report.
    It uses the lxml library for robust XML handling and invokes the Arelle command-line
    tool for official validation.

    Workflow:
    1.  Configuration: Sets up all necessary file paths and parameters.
    2.  Data Extraction: Reads a mapping file and data from an Excel workbook.
    3.  XBRL Generation (with lxml): Loads a source XBRL template, updates dates and
        financial values, and saves the new file, correctly preserving all namespaces.
    4.  Arelle Validation: Invokes the Arelle command-line to perform a full, offline
        taxonomy validation on the newly generated file.
    5.  Reporting & Packaging: Reports on the validation results and creates a final
        .zip archive if the report is valid.

    Prerequisites:
    - This script must be run after the setup cells (Installation, Drive Mount,
      Imports, and Unzipping) have all completed successfully.
    - lxml must be installed, and `from lxml import etree as ET` must be in the Imports cell.
    - Source Excel and XBRL files must be available in the specified locations.
    """
    print("--- Starting Main Script ---")

    # --- Initialize an empty list to store log messages ---
    change_log = [f"Modification process started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"]

    # --- Check Arelle Version ---
    print("\n Checking Arelle version...")
    version_command = ["arelleCmdLine", "--version"]
    result = subprocess.run(version_command, capture_output=True, text=True, check=True)
    # The version info is usually in the first line of the standard output
    version_info = result.stdout.strip().split('\n')[0]
    print(f"  -> {version_info}")
    # Add the version to our log file for record-keeping
    change_log.append(version_info)

    # --- [2] Define File Paths and Check Taxonomy Versions ---
    drive_path = "/content/drive/MyDrive/BDE_REPORTING_SCRIPT/XBRL_Taxonomies/"
    taxonomy_zip_files = [
        drive_path + 'es-bde.zip',
        drive_path + 'es-bde-aux.zip',
        drive_path + 'eu-eurofiling.zip',
        drive_path + 'srf-eac_full_taxonomy_0.zip',
        drive_path + 'Full_taxonomy_release_4.1.zip'
    ]

    # --- NEW: Check the last modified date of each taxonomy package ---
    print("\n[2] Checking taxonomy package versions (last modified date)...")
    change_log.append("\n[TAXONOMY VERSIONS USED]")
    for zip_file_path in taxonomy_zip_files:
        if os.path.exists(zip_file_path):
            mod_time = os.path.getmtime(zip_file_path)
            mod_date_str = datetime.fromtimestamp(mod_time).strftime('%Y-%m-%d %H:%M:%S')
            log_message = f"- {os.path.basename(zip_file_path)}: Last Modified {mod_date_str}"
            print(log_message)
            change_log.append(log_message)
        else:
            print(f"- WARNING: Could not find taxonomy package: {os.path.basename(zip_file_path)}")

    # --- Define All File Paths and Parameters ---
    drive_path = "/content/drive/MyDrive/BDE_REPORTING_SCRIPT/XBRL_Taxonomies/"
    local_taxonomy_folder = 'local_taxonomies'
    excel_file = 'EP_4_MAY25.xlsx'
    source_xbrl_file = 'ES6885_ES_EP_EDE_EPM1_20250430_20250506.xbrl'
    data_sheet_name = 'EP_4-2'
    mapping_sheet_name = 'mapping_ep4_2'

    try:
        # --- [1] Read Mapping and Data from Excel ---
        print(f"\n[1] Reading data and mapping from '{excel_file}'...")
        mapping_df = pd.read_excel(excel_file, sheet_name=mapping_sheet_name)

        mapping_dict = dict(zip(mapping_df['Excel_Code'].astype(str), mapping_df['XBRL_Context_Ref']))
        codes_to_find = set(mapping_dict.keys())

        workbook = openpyxl.load_workbook(excel_file, data_only=True)
        data_sheet = workbook[data_sheet_name]

        label_row_index = -1; max_hits = 0
        for i, row in enumerate(data_sheet.iter_rows()):
            hits_in_row = sum(1 for cell in row if re.search(r'\(([a-z])\)', str(cell.value)) and re.search(r'\(([a-z])\)', str(cell.value)).group(1) in codes_to_find)
            if hits_in_row > max_hits: max_hits = hits_in_row; label_row_index = i + 1
        if label_row_index == -1: raise ValueError("Could not identify the main label row in the Excel sheet.")

        new_data_values = {}
        label_row = list(data_sheet.iter_rows(min_row=label_row_index, max_row=label_row_index))[0]
        for cell in label_row:
            match = re.search(r'\(([a-z])\)', str(cell.value))
            if match and match.group(1) in mapping_dict:
                letter_code = match.group(1)
                data_cell = data_sheet.cell(row=cell.row + 1, column=cell.column + 2)
                new_data_values[letter_code] = data_cell.value
        print("Data extraction complete.")

        # --- [2] Generate the Candidate XBRL File using lxml ---
        print(f"\n[2] Generating candidate XBRL file using lxml...")

        # Use lxml's parser. This correctly handles all namespace information.
        parser = ET.XMLParser(remove_blank_text=True)
        tree = ET.parse(source_xbrl_file, parser)
        root = tree.getroot()
        # Get the namespace map from the parsed document
        ns_map = root.nsmap

        # Update dates and log the change
        original_date_str = root.find('.//xbrli:instant', namespaces=ns_map).text
        last_day_of_next_month = (date.fromisoformat(original_date_str).replace(day=1) + relativedelta(months=1))
        last_day_of_next_month = last_day_of_next_month.replace(day=calendar.monthrange(last_day_of_next_month.year, last_day_of_next_month.month)[1])
        new_date_str = last_day_of_next_month.isoformat()

        # --- Log the date change before making it ---
        if original_date_str != new_date_str:
             change_log.append(f"\n[DATE UPDATE]\n- Changed report date from '{original_date_str}' to '{new_date_str}'")

        for date_element in root.findall('.//xbrli:instant', namespaces=ns_map):
            date_element.text = new_date_str

        # Update facts and log each change
        for letter_code, context_ref in mapping_dict.items():
            if letter_code in new_data_values and new_data_values[letter_code] is not None:
                new_value = new_data_values[letter_code]
                fact_to_update = root.find(f".//*[@contextRef='{context_ref}']", namespaces=ns_map)
                if fact_to_update is not None:
                    old_value = fact_to_update.text
                    # --- Log the change if the value is different ---
                    if str(old_value) != str(new_value):
                        log_message = f"- Context '{context_ref}' (Code: {letter_code}): Changed value from '{old_value}' to '{new_value}'"
                        change_log.append(log_message)
                    # Make the change
                    fact_to_update.text = str(new_value)

        # Generate new filename
        reported_date_formatted = last_day_of_next_month.strftime('%Y%m%d')
        today_formatted = datetime.now().strftime('%Y%m%d')
        new_filename = f"ES6885_ES_EP_EDE_EPM1_{reported_date_formatted}_{today_formatted}.xbrl"

        # Write the file using lxml. This preserves all namespaces correctly.
        tree.write(new_filename, pretty_print=True, xml_declaration=True, encoding='UTF-8')
        print(f"Candidate file '{new_filename}' saved to Colab session.")

        # --- [3] Validate via Arelle Command Line Interface ---
        print(f"\n[3] Starting taxonomy validation with Arelle command line...")
        command = [
            "arelleCmdLine", "--file", new_filename, "--validate",
            "--remap-url", f"http://www.bde.es/|{os.path.join(local_taxonomy_folder, 'www.bde.es/')}",
            "--remap-url", f"http://www.eba.europa.eu/|{os.path.join(local_taxonomy_folder, 'www.eba.europa.eu/')}",
            "--remap-url", f"http://www.eurofiling.info/|{os.path.join(local_taxonomy_folder, 'www.eurofiling.info/')}",
            "--logFile", "arelle_validation_log.txt", "--logFormat", "text"
        ]
        print("Running Arelle...")
        subprocess.run(command)

        # --- [4] Read and display the Arelle report ---
        print("\n--- Arelle Validation Report ---")
        validation_successful = False
        with open("arelle_validation_log.txt", "r") as log_file:
            log_contents = log_file.read()

        if '[error]' not in log_contents.lower():
            print("✅ TAXONOMY VALIDATION SUCCESS: No critical errors reported.")
            validation_successful = True
        else:
            print("❌ TAXONOMY VALIDATION FAILED: Errors were reported by Arelle.")
            print("\nFull Log:")
            print(log_contents.strip())

        # --- [5] Create the Final .ZIP Archive if validation passed ---
        if validation_successful:
            print(f"\n[5] Creating final .zip archive...")
            base_name = os.path.splitext(new_filename)[0]
            output_zip_filename = base_name + ".zip"
            with zipfile.ZipFile(output_zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
                zipf.write(new_filename, arcname=os.path.basename(new_filename))
            print(f"✅ Success! Created '{output_zip_filename}'")

        # --- STEP 5: WRITE THE MODIFICATION LOG FILE ---
        log_filename = "modification_log.txt"
        print(f"\n[5] Writing modification log to '{log_filename}'...")
        with open(log_filename, "w") as f:
            for line in change_log:
                f.write(line + "\n")
        print("✅ Log file created successfully.")

        print("\n--- PROCESS COMPLETE ---")

    except FileNotFoundError as fnf_error:
        print(f"\nFILE NOT FOUND ERROR: {fnf_error}. Please ensure all source files are available.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

# --- Execute the main function ---
if __name__ == "__main__":
    try:
        main()
    except NameError as ne:
        print(f"\nSCRIPT SETUP ERROR: A required library was not imported. Please run the 'Import Libraries' cell first. Details: {ne}")
    except Exception as e:
        print(f"\nA critical error occurred: {e}")

--- Starting Main Script ---

 Checking Arelle version...
  -> Arelle(r) 2.37.28.dev9+gcf964068 (64bit)

[2] Checking taxonomy package versions (last modified date)...
- es-bde.zip: Last Modified 2025-06-19 10:58:56
- es-bde-aux.zip: Last Modified 2025-06-19 09:42:35
- eu-eurofiling.zip: Last Modified 2025-06-19 09:39:57
- srf-eac_full_taxonomy_0.zip: Last Modified 2025-06-19 09:44:15
- Full_taxonomy_release_4.1.zip: Last Modified 2025-06-19 09:51:48

[1] Reading data and mapping from 'EP_4_MAY25.xlsx'...
Data extraction complete.

[2] Generating candidate XBRL file using lxml...
Candidate file 'ES6885_ES_EP_EDE_EPM1_20250531_20250625.xbrl' saved to Colab session.

[3] Starting taxonomy validation with Arelle command line...
Running Arelle...

--- Arelle Validation Report ---

FILE NOT FOUND ERROR: [Errno 2] No such file or directory: 'arelle_validation_log.txt'. Please ensure all source files are available.
